In [1]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
import datetime
from datetime import timedelta
import logging
#from node_rules import cgcr_oxygen_tree, cgcr_aluminium_prediction, vag_aluminium_prediction

warnings.filterwarnings('ignore')

# for logging purposes
now = datetime.datetime.now()
print("Start Time : ", now)


Start Time :  2022-02-23 11:10:18.836173


In [6]:
# importing heat_analysis
#heat_analysis = pd.read_sql("select TOP(1000) * from heat_analysis order by MSG_TIME_STAMP desc", conn)

heat_analysis = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\input\DeS Input.xlsx', sheet_name='heat analysis')

# importing lf_heat_data
lf_heat_data = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\input\DeS Input.xlsx', sheet_name='lf data')

# importing the grade mapping fact table
grade_mapping = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\input\DeS Input.xlsx', sheet_name='grade mapping')

actual_al = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\input\DeS Input.xlsx', sheet_name='result')

#nodes = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\input\DeS Input.xlsx', sheet_name='nodes')

######################### data processing ##############################

print(len(heat_analysis))
print(len(lf_heat_data))

# calculating required columns
lf_heat_data['final_O2_ppm'] = lf_heat_data['TAP_O2'] + lf_heat_data['O2AFTERCELOX']

# converting ALBAR to tons
lf_heat_data['ALBAR'] = lf_heat_data['ALBAR']/19

# renaming the required columns
lf_heat_data.rename(columns = {
    'GRADE_TYPE':'GRADE',
    'LM_START_WT':'LM_Start_Wt_lf',
    'FIRSTMEASTEMP':'1st_Probe_temp_lf',
    'ALBAR':'Al_Bar_lf',
    'LIME':'Lime_tap',
    'SIMN':'SiMn_tap',
    'LTA':'LTA_lf'
}, inplace = True)

# keeping only the required rows
lf_heat_data = lf_heat_data.groupby(['HEAT_NUMBER']).agg({'MSG_TIME_STAMP':'max','GRADE': 'first','1st_Probe_temp_lf':'max', 'Al_Bar_lf':'max','LM_Start_Wt_lf':'max','LTA_lf':'max','Lime_tap':'max','SiMn_tap':'max','final_O2_ppm':'max'})
lf_heat_data.reset_index(inplace = True)
lf_heat_data.sort_values(by = ['MSG_TIME_STAMP'], ascending = False, inplace = True)
lf_heat_data.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)

# preprocessing heat_analysis
# if there are multiple records for a heat, keeping only the latest one
heat_analysis.drop_duplicates(subset = ['HEAT_NUMBER'], keep = 'first', inplace = True)

# keeping only required columns
heat_analysis = heat_analysis[['HEAT_NUMBER','AGGREGATE','AL_TOTAL','S','C','SI','Month']]

# renaming columns as per requirement
heat_analysis.rename(columns = {'AL_TOTAL':'AL_chem_first', 'S':'S_chem_first','SI':'SI_chem_first','C':'C_chem_first'}, inplace = True)


# merging sample chemistry data with lf data
ads = heat_analysis.merge(lf_heat_data, on = ['HEAT_NUMBER'], how = 'inner')
print(len(ads))

ads['LOI_perc'] = 6

# mapping the grades to their grade type (CG/CR or VAG)
# the grades that don't get mapped are billet heats, and so will not be run in this model
ads = ads.merge(grade_mapping, on = ['GRADE'], how = 'inner')

print(len(ads))

13642
36440
9265
9261


In [8]:
ads_test=ads.copy()
#ads_test=ads_test[(ads_test['Month']==12) | (ads_test['Month']==1) | (ads_test['Month']==2)]
 
ads_test=ads_test.merge(nodes, on ='HEAT_NUMBER', how ='left')
print(len(ads_test))
ads_test=ads_test[ads_test['GRADE_TYPE']=='CG/CR']
print(len(ads_test))


9326
4969
Carbon_Node_2
0     1488
1      499
2      461
3      628
4       48
5      559
6      185
7      164
8      121
9       55
10     187
11     194
12     107
13     109
14     108
15      42
16      14
Name: HEAT_NUMBER, dtype: int64
4969


In [9]:
model_ads=ads_test.copy()
print(len(model_ads))

model_ads=model_ads[(model_ads['final_O2_ppm']>700) & (model_ads['final_O2_ppm']<=1200)]
print(len(model_ads))
model_ads=model_ads[(model_ads['LM_Start_Wt_lf']>=170) & (model_ads['LM_Start_Wt_lf']<=205)]
print(len(model_ads))
model_ads=model_ads[(model_ads['Al_Bar_lf']<=25) & (model_ads['Al_Bar_lf']>=15)]
print(len(model_ads))
#print(len(model_ads[model_ads['Lime_tap']<=750]))
#print(len(model_ads[model_ads['AL_chem_first']<=750]))
#print(len(model_ads[model_ads['S_chem_first']<=750]))

4969
4123
3793
3428


In [ ]:
#actual_al = pd.read_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 22\input\DeS Input.xlsx', sheet_name='result')


In [10]:

def outlier_treatment(df,col_list,lv_imp,uv_imp):
    """
    
    Intended to treat outlier values based on defined thresholds lv and uv
    returns : dataframe with values treated for outliers
    """
    for col,lv,uv in zip(col_list,lv_imp,uv_imp):
        df[col+'_treated'] = [uv if x>uv else x for x in df[col]]
        df[col+'_treated'] = [lv if x<lv else x for x in df[col+'_treated']]
        
        #print(df[col+'_treated'])
    return df

def impute_missing(df,cols=[],simple_impute=True,impute_value=0):
    """
    
    Intended to impute missing values for the specified columns by the specified value
    returns : dataframe with imputed values
    """
    if simple_impute == 1:
            if len(cols)>0:
                df[cols] = df[cols].fillna(value=impute_value)
            else:
                df.fillna(impute_value,inplace=True)
    else:
        df.fillna(df[cols].median(),inplace=True)
        print("Column and Median to be replace : ",cols, df[cols].median())
    
    return df

def univariate(df):
    """
     
    Intended to provide a percentile distribution for numeric columns and 
    frenquency counts for categorical variables
    returns : data frame with univariates for each column
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','object']
    quantiles = [0,0.001,0.01,0.05,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9,0.95,0.99,0.999,1]
    
    df_num = pd.DataFrame(df.select_dtypes(include=numerics).quantile(quantiles).transpose())
    num_cols = df_num.index.tolist()
    char_cols = [cols for cols in df.columns if cols not in num_cols]
    
    """
    df_char = df[char_cols]
    df_cat_agg = pd.DataFrame()
    
    # Frequency counts for char columns 
    if len(char_cols) > 0:
        count = 0
        for col in char_cols:
            df_ = df_char.groupby(col).agg(['count'])
            df_['column_name'] = col
            df_cat_agg = df_cat_agg.append(df_)      
    
    #return df_num,df_cat_agg
    """
    
    return df_num

features = ['final_O2_ppm', 'LM_Start_Wt_lf',
           '1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap',
           'AL_chem_first', 'S_chem_first']

# model_ads=ads_test.copy()
model_ads=model_ads.merge(actual_al[['HEAT_NUMBER','Actual Al']].rename(columns={'Actual Al':'AL_mat'}),on='HEAT_NUMBER', how='left')
model_ads = model_ads[model_ads['AL_mat'].notna()]
#model_ads = model_ads[model_ads['AL_mat']!=0]

print(len(model_ads))
model_ads = model_ads[model_ads['AL_mat']<=600]
print(len(model_ads))


cols_null_value_treatment=['final_O2_ppm', 'LM_Start_Wt_lf',
           '1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap',
           'AL_chem_first', 'S_chem_first','AL_mat']

model_ads = impute_missing(model_ads, cols=cols_null_value_treatment,simple_impute=False)
print(len(model_ads))

# Outlier treatment
cols_of_interest=['AL_chem_first','S_chem_first','Lime_tap']
# feature_cols = [x for x in cols_of_interest if x not in ['Heat','AL_mat']]
univar_num = univariate(model_ads[['AL_chem_first','S_chem_first','Lime_tap']])


lv = univar_num[0.001].to_list()
uv = univar_num[0.99].to_list()
# Treat Outliers

model_ads = outlier_treatment(model_ads,cols_of_interest,lv,uv)

print(len(model_ads))

# features = ['final_O2_ppm', 'LM_Start_Wt_lf',
#            '1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap',
#            'AL_chem_first', 'S_chem_first']
# features = ['final_O2_ppm_treated', 'LM_Start_Wt_lf_treated',
#            '1st_Probe_temp_lf_treated','Al_Bar_lf_treated', 'Lime_tap_treated',
#            'AL_chem_first_treated', 'S_chem_first_treated']

#model_ads[['final_O2_ppm', 'LM_Start_Wt_lf','1st_Probe_temp_lf','Al_Bar_lf', 'Lime_tap','AL_chem_first', 'S_chem_first']]=model_ads[['final_O2_ppm_treated', 'LM_Start_Wt_lf_treated','1st_Probe_temp_lf_treated','Al_Bar_lf_treated', 'Lime_tap_treated','AL_chem_first_treated', 'S_chem_first_treated']]
model_ads[['Lime_tap','AL_chem_first', 'S_chem_first']]=model_ads[['Lime_tap_treated','AL_chem_first_treated', 'S_chem_first_treated']]



# Setting target variable
target = ['AL_mat']


3171
3140
Column and Median to be replace :  ['final_O2_ppm', 'LM_Start_Wt_lf', '1st_Probe_temp_lf', 'Al_Bar_lf', 'Lime_tap', 'AL_chem_first', 'S_chem_first', 'AL_mat'] final_O2_ppm          988.000000
LM_Start_Wt_lf        188.200000
1st_Probe_temp_lf    1573.000000
Al_Bar_lf              20.842105
Lime_tap             1124.000000
AL_chem_first           0.052000
S_chem_first            0.022000
AL_mat                194.350000
dtype: float64
3140
3140


In [17]:
model_ads_train_test_df=model_ads.copy()

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
# Split the training set to train and evaluate samples
X_train, X_test, y_train, y_test = train_test_split(model_ads_train_test_df[features],
                                                    model_ads_train_test_df[target],
                                                    test_size=0.25,
                                                    random_state=97)

#Merging the datasets
train_df = pd.concat([X_train,y_train], axis = 1).reset_index(drop=True)
test_df = pd.concat([X_test,y_test], axis = 1).reset_index(drop=True)

def train_decision_tree(df_features,df_target,n_=200,leaf_=10,split_=25,md_=4, random_state = 97):
    
    # Initialise dataframe 
    weights=pd.DataFrame(columns=['Variable','Importance_gain','iteration','rand_index'])
    
    # Instantiate the model with required parameters
    model_=DecisionTreeRegressor(max_depth=md_, random_state=random_state,criterion='mse',min_samples_leaf=leaf_, 
                                    min_samples_split=split_)    
        
    print(model_)
    model_.fit(df_features,df_target)
    weights = pd.DataFrame(model_.feature_importances_,
                                   index = df_features.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)   
    return model_,weights

def mape(y_pred,y_true,label,key): 
    mape_calc = pd.DataFrame(data=np.column_stack((y_pred,y_true,key)),columns=['predicted','actual','key'])
    mape_calc['mape'] = abs(mape_calc['predicted']-mape_calc['actual'])
    mape_calc = mape_calc[mape_calc['actual']>0]
    #mape_calc.to_csv(f"{output_path}\\"+label+f'_mape_validate_{now}.csv')
    
    return mape_calc['mape'].mean()


# Running Decision Tree
model_rf,weights_rf = train_decision_tree(X_train,y_train,leaf_=10,split_=25,md_=4,random_state = 97)

# Predicting on the train and test data
pred_train_rf = model_rf.predict(X_train)
pred_test_rf = model_rf.predict(X_test)

print("MAPE for train is {}ppm".format(round(mape(pred_train_rf,y_train.values,'train',y_train.index),2)))
print("MAPE for test is {}ppm".format(round(mape(pred_test_rf,y_test.values,'test',y_test.index),2))) 

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=4,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=25,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=97, splitter='best')
MAPE for train is 60.73ppm
MAPE for test is 63.21ppm


In [18]:
# 'equations' is used to print the rules for the model which can be plugged into the optimization code
import statsmodels.api as sm
equations = []
mapes = []
r2s = []

for nodes in model_ads['Carbon_Node_2'].unique():
    
    print('node',nodes)
    model_ads_node=model_ads[model_ads['Carbon_Node_2']==nodes]
    print('# heats',len(model_ads_node))
    
    # Put the target (housing value -- MEDV) in another DataFrame
    X = model_ads_node[features]
    y = model_ads_node[["AL_mat"]]

    # Note the difference in argument order
    model = sm.OLS(y, X).fit()

    predictions = model.predict(X) # make the predictions by the model
    y['AL_mat'][y['AL_mat']==0]=186

    # Mape calculation
    y_true, y_pred = np.array(y), np.array(predictions)
    print("MAPE : ", np.mean(np.abs((y_true - y_pred) / y_true)) * 100)
    mapes.append("MAPE : " + str(np.mean(np.abs((y_true - y_pred) / y_true)) * 100))

    # Print out the statistics
    #model.summary()

    # Calculating R Square
    r_squared_table = pd.read_html(model.summary().tables[0].as_html(),header=0,index_col=0)[0]
    r_squared_table.columns = ['1','2','3']
    try:
        adj_r_square = float(r_squared_table[r_squared_table['2']=='Adj. R-squared (uncentered):']['3']) * 100
    except:
        adj_r_square = float(r_squared_table[r_squared_table['2']=='Adj. R-squared:']['3']) * 100

    # Saving R2s
    r2s.append("Adjusted RSquared : " + str(adj_r_square))

    # Getting the coefficient table
    coefficient_table = pd.read_html(model.summary().tables[1].as_html(),header=0,index_col=0)[0]

    # Resetting the index
    coefficient_table = coefficient_table.reset_index()

    # Keeping columns of interest
    coefficient_table = coefficient_table[['index','coef']]

    equation = ''
    for i in range(coefficient_table.shape[0]):
        if i == 0:
            equation = str(coefficient_table['coef'][i]) + ' * ' + "row[\'" +str(coefficient_table['index'][i]) +"\']"
        else:
            equation = equation + " + " + str(coefficient_table['coef'][i]) + ' * ' + 'row[\'' + str(coefficient_table['index'][i]) + '\']'

    # saving equations
    equations.append("row['AL_mat_treated'] = " + equation)

    print("Adjusted R2 : ",adj_r_square)       
    print(equation)

node 0
# heats 1049
MAPE :  56.99569968651028
Adjusted R2 :  85.3
0.0335 * row['final_O2_ppm'] + 1.0215 * row['LM_Start_Wt_lf'] + 0.0041 * row['1st_Probe_temp_lf'] + -4.3121 * row['Al_Bar_lf'] + 0.0385 * row['Lime_tap'] + -1690.1376 * row['AL_chem_first'] + 4586.493 * row['S_chem_first']
node 3
# heats 402
MAPE :  60.34865595899691
Adjusted R2 :  83.89999999999999
0.0298 * row['final_O2_ppm'] + -0.256 * row['LM_Start_Wt_lf'] + 0.0465 * row['1st_Probe_temp_lf'] + 1.9415 * row['Al_Bar_lf'] + 0.0371 * row['Lime_tap'] + -1674.4821 * row['AL_chem_first'] + 5280.5625 * row['S_chem_first']
node 13
# heats 73
MAPE :  24.011604387606006
Adjusted R2 :  93.5
0.1723 * row['final_O2_ppm'] + -0.5903 * row['LM_Start_Wt_lf'] + -0.0425 * row['1st_Probe_temp_lf'] + -1.913 * row['Al_Bar_lf'] + 0.0173 * row['Lime_tap'] + 4189.3749 * row['AL_chem_first'] + 6710.7694 * row['S_chem_first']
node 4
# heats 32
MAPE :  81.56950601574498
Adjusted R2 :  84.6
0.0513 * row['final_O2_ppm'] + 0.0527 * row['LM_Start_Wt

In [20]:
model_ads.to_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\output\testing\model_testing.xlsx')

In [23]:
ads.to_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\output\testing\all_data_aug_feb.xlsx')

4969


In [252]:
def cgcr_aluminium_prediction_new(row):
    """

    This function will calculate the aluminium material addition for CG/CR grades
    # the "node" columns are only for debugging purposes
    """

    al_lsl = 0.030
    fade_rate = 0.004

    # initializing to 0, will get replaced if we use oxygen tree
    row['Oxygen Node'] = 0

    if (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] > 0.004) and (row['S_chem_first'] <= 0.02):

        row['Carbon Node'] = 1

        c=-0.0506 * row['final_O2_ppm'] + 0.5396 * row['LM_Start_Wt_lf'] + -0.0304 * row['1st_Probe_temp_lf'] + 3.1725 * row['Al_Bar_lf'] + 0.0902 * row['Lime_tap'] + -969.3439 * row['AL_chem_first'] + 377.9919 * row['S_chem_first'] 

        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] <= 0.022) and (row['Al_Bar_lf'] > 15.5):

            row['Oxygen Node'] = 1

            row['AL_mat']=-0.0506 * row['final_O2_ppm'] + 0.5396 * row['LM_Start_Wt_lf'] + -0.0304 * row['1st_Probe_temp_lf'] + 3.1725 * row['Al_Bar_lf'] + 0.0902 * row['Lime_tap'] + -969.3439 * row['AL_chem_first'] + 377.9919 * row['S_chem_first']

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] <= 0.022) and (row['Al_Bar_lf'] <= 15.5):

            row['Oxygen Node'] = 15

            o = 0.0333 * row['final_O2_ppm'] + 1.9003 * row['LM_Start_Wt_lf'] + 0.4419 * row['1st_Probe_temp_lf'] + -73.0713 * row['Al_Bar_lf'] + -0.025 * row['Lime_tap'] + 1918.4667 * row['AL_chem_first'] + 5078.7158 * row['S_chem_first']

            row['AL_mat'] = (c+o)/2

        else:

            row['AL_mat'] = c

#         row['AL_mat']=-0.0506 * row['final_O2_ppm'] + 0.5396 * row['LM_Start_Wt_lf'] + -0.0304 * row['1st_Probe_temp_lf'] + 3.1725 * row['Al_Bar_lf'] + 0.0902 * row['Lime_tap'] + -969.3439 * row['AL_chem_first'] + 377.9919 * row['S_chem_first']
#         row['AL_mat']=0.0513 * row['final_O2_ppm'] + 0.0527 * row['LM_Start_Wt_lf'] + 0.1355 * row['1st_Probe_temp_lf'] + -5.8581 * row['Al_Bar_lf'] + 0.0932 * row['Lime_tap'] + -4195.7562 * row['AL_chem_first'] + 7493.6819 * row['S_chem_first']
            
    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.021) and (row['SI_chem_first'] > 0.004):

        row['Carbon Node'] = 2

        row['AL_mat']=0.0298 * row['final_O2_ppm'] + -0.256 * row['LM_Start_Wt_lf'] + 0.0465 * row['1st_Probe_temp_lf'] + 1.9415 * row['Al_Bar_lf'] + 0.0371 * row['Lime_tap'] + -1674.4821 * row['AL_chem_first'] + 5280.5625 * row['S_chem_first']
            
    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] > 0.004) and (row['S_chem_first'] > 0.02):

        row['Carbon Node'] = 3
            
        row['AL_mat']=0.0298 * row['final_O2_ppm'] + -0.256 * row['LM_Start_Wt_lf'] + 0.0465 * row['1st_Probe_temp_lf'] + 1.9415 * row['Al_Bar_lf'] + 0.0371 * row['Lime_tap'] + -1674.4821 * row['AL_chem_first'] + 5280.5625 * row['S_chem_first']

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] <= 0.004) and (row['SiMn_tap'] <= 170.119):

        row['Carbon Node'] = 4

        row['AL_mat']=0.0513 * row['final_O2_ppm'] + 0.0527 * row['LM_Start_Wt_lf'] + 0.1355 * row['1st_Probe_temp_lf'] + -5.8581 * row['Al_Bar_lf'] + 0.0932 * row['Lime_tap'] + -4195.7562 * row['AL_chem_first'] + 7493.6819 * row['S_chem_first']

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.021) and (row['S_chem_first'] <= 0.029):

        row['Carbon Node'] = 5

        row['AL_mat']= 0.3657 * row['LM_Start_Wt_lf'] + 0.1113 * row['1st_Probe_temp_lf'] + -0.9966 * row['Al_Bar_lf'] + 0.0054 * row['Lime_tap'] + -2055.9061 * row['AL_chem_first'] + 4510.7115 * row['S_chem_first'] + 0.0002 * row['LTA_lf'] + -0.0406 * row['SiMn_tap'] + -3218.7582 * row['SI_chem_first'] + -854.3342 * row['C_chem_first']
            
    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] > 0.024):

        row['Carbon Node'] = 6

        c = 0.3412 * row['LM_Start_Wt_lf'] + 0.1243 * row['1st_Probe_temp_lf'] + -2.3149 * row['Al_Bar_lf'] + 0.0026 * row['Lime_tap'] + -707.5725 * row['AL_chem_first'] + 4396.2823 * row['S_chem_first'] + 0.0043 * row['LTA_lf'] + 0.0328 * row['SiMn_tap'] + -5089.7416 * row['SI_chem_first'] + -1601.3223 * row['C_chem_first']
        row['AL_mat'] = c
        row['AL_mat']=0.0298 * row['final_O2_ppm'] + -0.256 * row['LM_Start_Wt_lf'] + 0.0465 * row['1st_Probe_temp_lf'] + 1.9415 * row['Al_Bar_lf'] + 0.0371 * row['Lime_tap'] + -1674.4821 * row['AL_chem_first'] + 5280.5625 * row['S_chem_first']


    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] <= 0.024):

        row['Carbon Node'] = 7

        row['AL_mat']=-0.1593 * row['final_O2_ppm'] + 2.4897 * row['LM_Start_Wt_lf'] + -0.106 * row['1st_Probe_temp_lf'] + -0.3263 * row['Al_Bar_lf'] + 0.0438 * row['Lime_tap'] + 1741.0624 * row['AL_chem_first'] + 3372.9862 * row['S_chem_first']

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.021) and (row['SI_chem_first'] <= 0.004):

        row['Carbon Node'] = 8

            
#         row['AL_mat']=0.1605 * row['final_O2_ppm'] + 0.5668 * row['LM_Start_Wt_lf'] + -0.1534 * row['1st_Probe_temp_lf'] + 2.8439 * row['Al_Bar_lf'] + 0.0792 * row['Lime_tap'] + -670.0398 * row['AL_chem_first'] + 2995.0794 * row['S_chem_first']
        row['AL_mat']= 0.3657 * row['LM_Start_Wt_lf'] + 0.1113 * row['1st_Probe_temp_lf'] + -0.9966 * row['Al_Bar_lf'] + 0.0054 * row['Lime_tap'] + -2055.9061 * row['AL_chem_first'] + 4510.7115 * row['S_chem_first'] + 0.0002 * row['LTA_lf'] + -0.0406 * row['SiMn_tap'] + -3218.7582 * row['SI_chem_first'] + -854.3342 * row['C_chem_first']

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] <= 0.01) and (row['S_chem_first'] <= 0.02):

        row['Carbon Node'] = 9


        row['AL_mat']=-0.0248 * row['final_O2_ppm'] + 0.2478 * row['LM_Start_Wt_lf'] + 0.5271 * row['1st_Probe_temp_lf'] + -3.9937 * row['Al_Bar_lf'] + -0.0248 * row['Lime_tap'] + -17420.0 * row['AL_chem_first'] + -15290.0 * row['S_chem_first']
            
    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] <= 0.01) and (row['S_chem_first'] > 0.02):

        row['Carbon Node'] = 10

        row['AL_mat']=-0.1353 * row['final_O2_ppm'] + -0.1133 * row['LM_Start_Wt_lf'] + 0.3506 * row['1st_Probe_temp_lf'] + -6.8778 * row['Al_Bar_lf'] + -0.0286 * row['Lime_tap'] + -779.5998 * row['AL_chem_first'] + 6268.7614 * row['S_chem_first']
        row['AL_mat']=0.025 * row['final_O2_ppm'] + 0.7301 * row['LM_Start_Wt_lf'] + 0.0486 * row['1st_Probe_temp_lf'] + -0.0983 * row['Al_Bar_lf'] + 0.0351 * row['Lime_tap'] + -2198.3152 * row['AL_chem_first'] + 2711.7074 * row['S_chem_first']
            
    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.021) and (row['S_chem_first'] > 0.029):

        row['Carbon Node'] = 11

        c = 0.7856 * row['LM_Start_Wt_lf'] + -0.0129 * row['1st_Probe_temp_lf'] + 4.362 * row['Al_Bar_lf'] + -0.0659 * row['Lime_tap'] + -1992.4572 * row['AL_chem_first'] + 7477.7345 * row['S_chem_first'] + -0.0007 * row['LTA_lf'] + 0.0208 * row['SiMn_tap'] + -2220.0319 * row['SI_chem_first'] + -1081.9221 * row['C_chem_first']
        c=0.025 * row['final_O2_ppm'] + 0.7301 * row['LM_Start_Wt_lf'] + 0.0486 * row['1st_Probe_temp_lf'] + -0.0983 * row['Al_Bar_lf'] + 0.0351 * row['Lime_tap'] + -2198.3152 * row['AL_chem_first'] + 2711.7074 * row['S_chem_first']
        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] <= 0.03):

            row['Oxygen Node'] = 5

            o = 0.0472 * row['final_O2_ppm'] + 0.4099 * row['LM_Start_Wt_lf'] + 0.0752 * row['1st_Probe_temp_lf'] + -1.4064 * row['Al_Bar_lf'] + -0.0045 * row['Lime_tap'] + -2538.0108 * row['AL_chem_first'] + 3874.9522 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] > 0.03):

            row['Oxygen Node'] = 13

            o = -0.0843 * row['final_O2_ppm'] + 1.9108 * row['LM_Start_Wt_lf'] + 0.0295 * row['1st_Probe_temp_lf'] + -5.8736 * row['Al_Bar_lf'] + -0.0418 * row['Lime_tap'] + -757.535 * row['AL_chem_first'] + 3494.3731 * row['S_chem_first']

            row['AL_mat'] = o

        else:

            row['AL_mat'] = c
            
        row['AL_mat']=0.025 * row['final_O2_ppm'] + 0.7301 * row['LM_Start_Wt_lf'] + 0.0486 * row['1st_Probe_temp_lf'] + -0.0983 * row['Al_Bar_lf'] + 0.0351 * row['Lime_tap'] + -2198.3152 * row['AL_chem_first'] + 2711.7074 * row['S_chem_first']

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.016):

        row['Carbon Node'] = 12
        c = 0.1261 * row['LM_Start_Wt_lf'] + 0.1722 * row['1st_Probe_temp_lf'] + 1.0519 * row['Al_Bar_lf'] + -0.016 * row['Lime_tap'] + -1284.3628 * row['AL_chem_first'] + 4291.4388 * row['S_chem_first'] + -0.01 * row['LTA_lf'] + -0.0203 * row['SiMn_tap'] + -3953.4701 * row['SI_chem_first'] + -2227.1677 * row['C_chem_first']
        c=-0.0576 * row['final_O2_ppm'] + 2.3884 * row['LM_Start_Wt_lf'] + 0.0692 * row['1st_Probe_temp_lf'] + -1.1152 * row['Al_Bar_lf'] + -0.0448 * row['Lime_tap'] + -14040.0 * row['AL_chem_first'] + 3008.1748 * row['S_chem_first']
        if (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.024) and (row['final_O2_ppm'] <= 1016.35):

            row['Oxygen Node'] = 14

            o = 0.0056 * row['final_O2_ppm'] + 0.8489 * row['LM_Start_Wt_lf'] + 0.1301 * row['1st_Probe_temp_lf'] + -5.7387 * row['Al_Bar_lf'] + -0.0643 * row['Lime_tap'] + -3757.1869 * row['AL_chem_first'] + 5313.2647 * row['S_chem_first']

            row['AL_mat'] = o

        else:

            row['AL_mat'] = c

        row['AL_mat']=-0.0576 * row['final_O2_ppm'] + 2.3884 * row['LM_Start_Wt_lf'] + 0.0692 * row['1st_Probe_temp_lf'] + -1.1152 * row['Al_Bar_lf'] + -0.0448 * row['Lime_tap'] + -14040.0 * row['AL_chem_first'] + 3008.1748 * row['S_chem_first']
            
    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] > 0.024) and (row['AL_chem_first'] > 0.022):

        row['Carbon Node'] = 13
        c = 0.5743 * row['LM_Start_Wt_lf'] + 0.0822 * row['1st_Probe_temp_lf'] + -1.4686 * row['Al_Bar_lf'] + 0.0017 * row['Lime_tap'] + -1187.9995 * row['AL_chem_first'] + 4171.1597 * row['S_chem_first'] + 0.0015 * row['LTA_lf'] + -0.1217 * row['SiMn_tap'] + 1952.4868 * row['SI_chem_first'] + -1138.2887 * row['C_chem_first']
        c=0.1723 * row['final_O2_ppm'] + -0.5903 * row['LM_Start_Wt_lf'] + -0.0425 * row['1st_Probe_temp_lf'] + -1.913 * row['Al_Bar_lf'] + 0.0173 * row['Lime_tap'] + 4189.3749 * row['AL_chem_first'] + 6710.7694 * row['S_chem_first']
        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] <= 0.03):

            row['Oxygen Node'] = 5

            o = 0.0472 * row['final_O2_ppm'] + 0.4099 * row['LM_Start_Wt_lf'] + 0.0752 * row['1st_Probe_temp_lf'] + -1.4064 * row['Al_Bar_lf'] + -0.0045 * row['Lime_tap'] + -2538.0108 * row['AL_chem_first'] + 3874.9522 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] > 0.03):

            row['Oxygen Node'] = 13

            o = -0.0843 * row['final_O2_ppm'] + 1.9108 * row['LM_Start_Wt_lf'] + 0.0295 * row['1st_Probe_temp_lf'] + -5.8736 * row['Al_Bar_lf'] + -0.0418 * row['Lime_tap'] + -757.535 * row['AL_chem_first'] + 3494.3731 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.024) and (row['final_O2_ppm'] > 1016.35):

            row['Oxygen Node'] = 10

            o = 0.0172 * row['final_O2_ppm'] + 0.7945 * row['LM_Start_Wt_lf'] + 0.0299 * row['1st_Probe_temp_lf'] + -0.0299 * row['Al_Bar_lf'] + -0.0134 * row['Lime_tap'] + -2280.7642 * row['AL_chem_first'] + 4619.1716 * row['S_chem_first']

            row['AL_mat'] = min(c,o) - 33

        elif (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.024) and (row['final_O2_ppm'] <= 1016.35):

            row['Oxygen Node'] = 14

            o = 0.0056 * row['final_O2_ppm'] + 0.8489 * row['LM_Start_Wt_lf'] + 0.1301 * row['1st_Probe_temp_lf'] + -5.7387 * row['Al_Bar_lf'] + -0.0643 * row['Lime_tap'] + -3757.1869 * row['AL_chem_first'] + 5313.2647 * row['S_chem_first']

            row['AL_mat'] = min(c,o) - 33

        else:

            row['AL_mat'] = c

        row['AL_mat']=0.1723 * row['final_O2_ppm'] + -0.5903 * row['LM_Start_Wt_lf'] + -0.0425 * row['1st_Probe_temp_lf'] + -1.913 * row['Al_Bar_lf'] + 0.0173 * row['Lime_tap'] + 4189.3749 * row['AL_chem_first'] + 6710.7694 * row['S_chem_first']

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] > 0.024) and (row['AL_chem_first'] <= 0.022):
        row['Carbon Node'] = 14
        c = -0.1188 * row['LM_Start_Wt_lf'] + 0.2324 * row['1st_Probe_temp_lf'] + 7.6551 * row['Al_Bar_lf'] + -0.1695 * row['Lime_tap'] + 5022.7992 * row['AL_chem_first'] + 666.7963 * row['S_chem_first'] + -0.0046 * row['LTA_lf'] + 0.1344 * row['SiMn_tap'] + -10060.0 * row['SI_chem_first'] + -2723.9295 * row['C_chem_first']
        c=-0.0938 * row['final_O2_ppm'] + 0.7608 * row['LM_Start_Wt_lf'] + 0.197 * row['1st_Probe_temp_lf'] + -0.398 * row['Al_Bar_lf'] + -0.0564 * row['Lime_tap'] + -775.0687 * row['AL_chem_first'] + 2364.0028 * row['S_chem_first']
        o, node = cgcr_oxygen_tree(row)

        row['Oxygen Node'] = node
        row['AL_mat'] = min(c,o)

        row['AL_mat'] =-0.0938 * row['final_O2_ppm'] + 0.7608 * row['LM_Start_Wt_lf'] + 0.197 * row['1st_Probe_temp_lf'] + -0.398 * row['Al_Bar_lf'] + -0.0564 * row['Lime_tap'] + -775.0687 * row['AL_chem_first'] + 2364.0028 * row['S_chem_first']
        
    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] <= 0.004) and (row['SiMn_tap'] > 170.119):

        row['Carbon Node'] = 15
        o, node = cgcr_oxygen_tree(row)

        row['Oxygen Node'] = node
        row['AL_mat'] = o

        row['AL_mat']=-0.0477 * row['final_O2_ppm'] + 2.0904 * row['LM_Start_Wt_lf'] + -0.0394 * row['1st_Probe_temp_lf'] + -15.5702 * row['Al_Bar_lf'] + 0.0147 * row['Lime_tap'] + 404.7696 * row['AL_chem_first'] + 8637.0034 * row['S_chem_first']
        
    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] <= 0.016):

        row['Carbon Node'] = 16

        c = -1.0917 * row['LM_Start_Wt_lf'] + 0.5701 * row['1st_Probe_temp_lf'] + -21.2452 * row['Al_Bar_lf'] + -0.0651 * row['Lime_tap'] + 476.5523 * row['AL_chem_first'] + 6669.1629 * row['S_chem_first'] + -0.0036 * row['LTA_lf'] + 0.1242 * row['SiMn_tap'] + -5196.6844 * row['SI_chem_first'] + -2299.668 * row['C_chem_first']
        c=-0.9625 * row['final_O2_ppm'] + -0.9033 * row['LM_Start_Wt_lf'] + -0.08 * row['1st_Probe_temp_lf'] + 30.5226 * row['Al_Bar_lf'] + 0.3522 * row['Lime_tap'] + -13170.0 * row['AL_chem_first'] + 53750.0 * row['S_chem_first']
        o, node = cgcr_oxygen_tree(row)

        row['Oxygen Node'] = node
        row['AL_mat'] = min(c,o)
        
        row['AL_mat']=-0.9625 * row['final_O2_ppm'] + -0.9033 * row['LM_Start_Wt_lf'] + -0.08 * row['1st_Probe_temp_lf'] + 30.5226 * row['Al_Bar_lf'] + 0.3522 * row['Lime_tap'] + -13170.0 * row['AL_chem_first'] + 53750.0 * row['S_chem_first']

    else:
        print("weak node")
        row['AL_mat']=0.0335 * row['final_O2_ppm'] + 1.0215 * row['LM_Start_Wt_lf'] + 0.0041 * row['1st_Probe_temp_lf'] + -4.3121 * row['Al_Bar_lf'] + 0.0385 * row['Lime_tap'] + -1690.1376 * row['AL_chem_first'] + 4586.493 * row['S_chem_first']
        row['Carbon Node'] = 0
        row['Oxygen Node'] = 0
        row['AL_mat'] = (10.0 * (row['LM_Start_Wt_lf'] + 1) * (al_lsl - row['AL_chem_first'] + 16 * fade_rate ))/0.6

    return [row['AL_mat'], row['Carbon Node'], row['Oxygen Node']]

In [257]:
def cgcr_aluminium_prediction_original(row):
    """

    This function will calculate the aluminium material addition for CG/CR grades
    # the "node" columns are only for debugging purposes
    """

    al_lsl = 0.030
    fade_rate = 0.004

    # initializing to 0, will get replaced if we use oxygen tree
    row['Oxygen Node'] = 0

    if (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] > 0.004) and (row['S_chem_first'] <= 0.02):

        row['Carbon Node'] = 1

        c = 0.2798 * row['LM_Start_Wt_lf'] + 0.1403 * row['1st_Probe_temp_lf'] + -5.0366 * row['Al_Bar_lf'] + -0.0289 * row['Lime_tap'] + -764.6388 * row['AL_chem_first'] + 2453.4545 * row['S_chem_first'] + -0.0004 * row['LTA_lf'] + 0.0223 * row['SiMn_tap'] + -4409.1428 * row['SI_chem_first'] + 43.3277 * row['C_chem_first']


        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] <= 0.022) and (row['Al_Bar_lf'] > 15.5):

            row['Oxygen Node'] = 1

            row['AL_mat'] = 0.2798 * row['LM_Start_Wt_lf'] + 0.1403 * row['1st_Probe_temp_lf'] + -5.0366 * row['Al_Bar_lf'] + -0.0289 * row['Lime_tap'] + -764.6388 * row['AL_chem_first'] + 2453.4545 * row['S_chem_first'] + -0.0004 * row['LTA_lf'] + 0.0223 * row['SiMn_tap'] + -4409.1428 * row['SI_chem_first'] + 43.3277 * row['C_chem_first'] - 16

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] <= 0.022) and (row['Al_Bar_lf'] <= 15.5):

            row['Oxygen Node'] = 15

            o = 0.0333 * row['final_O2_ppm'] + 1.9003 * row['LM_Start_Wt_lf'] + 0.4419 * row['1st_Probe_temp_lf'] + -73.0713 * row['Al_Bar_lf'] + -0.025 * row['Lime_tap'] + 1918.4667 * row['AL_chem_first'] + 5078.7158 * row['S_chem_first']

            row['AL_mat'] = (c+o)/2

        else:

            row['AL_mat'] = c


    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.021) and (row['SI_chem_first'] > 0.004):

        row['Carbon Node'] = 2

        c = 0.5261 * row['LM_Start_Wt_lf'] + 0.0791 * row['1st_Probe_temp_lf'] + -1.3498 * row['Al_Bar_lf'] + -0.0258 * row['Lime_tap'] + -1708.4114 * row['AL_chem_first'] + 5901.3276 * row['S_chem_first'] + -1.061e-05 * row['LTA_lf'] + -0.0293 * row['SiMn_tap'] + -4352.8933 * row['SI_chem_first'] + -314.14 * row['C_chem_first']


        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] > 0.035):

            row['Oxygen Node'] = 2

            o = -0.0061 * row['final_O2_ppm'] + 0.7951 * row['LM_Start_Wt_lf'] + 0.067 * row['1st_Probe_temp_lf'] + -2.6335 * row['Al_Bar_lf'] + -0.0373 * row['Lime_tap'] + -1883.7621 * row['AL_chem_first'] + 4583.8493 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] <= 0.035):

            row['Oxygen Node'] = 3

            o = 0.0394 * row['final_O2_ppm'] + 0.4446 * row['LM_Start_Wt_lf'] + 0.0591 * row['1st_Probe_temp_lf'] + -1.131 * row['Al_Bar_lf'] + -0.0342 * row['Lime_tap'] + -1805.4052 * row['AL_chem_first'] + 4872.0171 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        else:

            row['AL_mat'] = c

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] > 0.004) and (row['S_chem_first'] > 0.02):

        row['Carbon Node'] = 3

        c = 0.0285 * row['LM_Start_Wt_lf'] + 0.1877 * row['1st_Probe_temp_lf'] + -3.4885 * row['Al_Bar_lf'] + -0.0269 * row['Lime_tap'] + -2018.3323 * row['AL_chem_first'] + 4606.0664 * row['S_chem_first'] + -0.0003 * row['LTA_lf'] + 0.0123 * row['SiMn_tap'] + -4181.5531 * row['SI_chem_first'] + -268.2512 * row['C_chem_first']


        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] > 0.022) and (row['AL_chem_first'] <= 0.059):

            row['Oxygen Node'] = 7

            o = 0.0587 * row['final_O2_ppm'] + -0.0184 * row['LM_Start_Wt_lf'] + 0.007 * row['1st_Probe_temp_lf'] + -2.7345 * row['Al_Bar_lf'] + -0.0025 * row['Lime_tap'] + 818.841 * row['AL_chem_first'] + 4921.6207 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        else:

            row['AL_mat'] = c

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] <= 0.004) and (row['SiMn_tap'] <= 170.119):

        row['Carbon Node'] = 4

        c = 0.855 * row['LM_Start_Wt_lf'] + 0.0838 * row['1st_Probe_temp_lf'] + -1.3642 * row['Al_Bar_lf'] + -0.036 * row['Lime_tap'] + -1413.6468 * row['AL_chem_first'] + 4535.899 * row['S_chem_first'] + -0.0013 * row['LTA_lf'] + 0.1752 * row['SiMn_tap'] + -17950.0 * row['SI_chem_first'] + -1058.9498 * row['C_chem_first']


        if(row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] > 0.022) and (row['AL_chem_first'] > 0.059):

            row['Oxygen Node'] = 6

            o = -0.0099 * row['final_O2_ppm'] + 0.7622 * row['LM_Start_Wt_lf'] + 0.0607 * row['1st_Probe_temp_lf'] + -3.3849 * row['Al_Bar_lf'] + -0.0656 * row['Lime_tap'] + -1057.3001 * row['AL_chem_first'] + 4924.1931 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] > 0.022) and (row['AL_chem_first'] <= 0.059):

            row['Oxygen Node'] = 7

            o = 0.0587 * row['final_O2_ppm'] + -0.0184 * row['LM_Start_Wt_lf'] + 0.007 * row['1st_Probe_temp_lf'] + -2.7345 * row['Al_Bar_lf'] + -0.0025 * row['Lime_tap'] + 818.841 * row['AL_chem_first'] + 4921.6207 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] > 0.05) and (row['S_chem_first'] <= 0.022) and (row['Al_Bar_lf'] <= 15.5):

            row['Oxygen Node'] = 15

            o = 0.0333 * row['final_O2_ppm'] + 1.9003 * row['LM_Start_Wt_lf'] + 0.4419 * row['1st_Probe_temp_lf'] + -73.0713 * row['Al_Bar_lf'] + -0.025 * row['Lime_tap'] + 1918.4667 * row['AL_chem_first'] + 5078.7158 * row['S_chem_first']

            row['AL_mat'] = o

        else:

            row['AL_mat'] = c


        row['AL_mat'] = 0.855 * row['LM_Start_Wt_lf'] + 0.0838 * row['1st_Probe_temp_lf'] + -1.3642 * row['Al_Bar_lf'] + -0.036 * row['Lime_tap'] + -1413.6468 * row['AL_chem_first'] + 4535.899 * row['S_chem_first'] + -0.0013 * row['LTA_lf'] + 0.1752 * row['SiMn_tap'] + -17950.0 * row['SI_chem_first'] + -1058.9498 * row['C_chem_first']

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.021) and (row['S_chem_first'] <= 0.029):

        row['Carbon Node'] = 5

        c = 0.3657 * row['LM_Start_Wt_lf'] + 0.1113 * row['1st_Probe_temp_lf'] + -0.9966 * row['Al_Bar_lf'] + 0.0054 * row['Lime_tap'] + -2055.9061 * row['AL_chem_first'] + 4510.7115 * row['S_chem_first'] + 0.0002 * row['LTA_lf'] + -0.0406 * row['SiMn_tap'] + -3218.7582 * row['SI_chem_first'] + -854.3342 * row['C_chem_first']

        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] > 0.035):

            row['Oxygen Node'] = 2

            o = -0.0061 * row['final_O2_ppm'] + 0.7951 * row['LM_Start_Wt_lf'] + 0.067 * row['1st_Probe_temp_lf'] + -2.6335 * row['Al_Bar_lf'] + -0.0373 * row['Lime_tap'] + -1883.7621 * row['AL_chem_first'] + 4583.8493 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] <= 0.035):

            row['Oxygen Node'] = 3

            o = 0.0394 * row['final_O2_ppm'] + 0.4446 * row['LM_Start_Wt_lf'] + 0.0591 * row['1st_Probe_temp_lf'] + -1.131 * row['Al_Bar_lf'] + -0.0342 * row['Lime_tap'] + -1805.4052 * row['AL_chem_first'] + 4872.0171 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] <= 0.03):

            row['Oxygen Node'] = 5

            o = 0.0472 * row['final_O2_ppm'] + 0.4099 * row['LM_Start_Wt_lf'] + 0.0752 * row['1st_Probe_temp_lf'] + -1.4064 * row['Al_Bar_lf'] + -0.0045 * row['Lime_tap'] + -2538.0108 * row['AL_chem_first'] + 3874.9522 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        else:
            row['AL_mat'] = c

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] > 0.024):

        row['Carbon Node'] = 6

        c = 0.3412 * row['LM_Start_Wt_lf'] + 0.1243 * row['1st_Probe_temp_lf'] + -2.3149 * row['Al_Bar_lf'] + 0.0026 * row['Lime_tap'] + -707.5725 * row['AL_chem_first'] + 4396.2823 * row['S_chem_first'] + 0.0043 * row['LTA_lf'] + 0.0328 * row['SiMn_tap'] + -5089.7416 * row['SI_chem_first'] + -1601.3223 * row['C_chem_first']

        row['AL_mat'] = c


    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] <= 0.024):

        row['Carbon Node'] = 7

        c = 0.3115 * row['LM_Start_Wt_lf'] + 0.1151 * row['1st_Probe_temp_lf'] + -0.3084 * row['Al_Bar_lf'] + 0.0033 * row['Lime_tap'] + -2436.6049 * row['AL_chem_first'] + 5896.8806 * row['S_chem_first'] + -0.0009 * row['LTA_lf'] + -0.0042 * row['SiMn_tap'] + -5288.325 * row['SI_chem_first'] + -1067.7573 * row['C_chem_first']

        if (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] > 0.015):

            row['Oxygen Node'] = 4

            o = 0.021 * row['final_O2_ppm'] + 0.5297 * row['LM_Start_Wt_lf'] + 0.054 * row['1st_Probe_temp_lf'] + -0.3041 * row['Al_Bar_lf'] + -0.004 * row['Lime_tap'] + -3664.7478 * row['AL_chem_first'] + 5646.8733 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        else:

            row['AL_mat'] = c


    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.021) and (row['SI_chem_first'] <= 0.004):

        row['Carbon Node'] = 8

        c = 0.609 * row['LM_Start_Wt_lf'] + 0.1795 * row['1st_Probe_temp_lf'] + -4.7042 * row['Al_Bar_lf'] + -0.0518 * row['Lime_tap'] + -1385.7228 * row['AL_chem_first'] + 2071.0482 * row['S_chem_first'] + 0.0011 * row['LTA_lf'] + 0.2036 * row['SiMn_tap'] + -23180.0 * row['SI_chem_first'] + 466.3279 * row['C_chem_first']

        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] > 0.035):

            row['Oxygen Node'] = 2

            o = -0.0061 * row['final_O2_ppm'] + 0.7951 * row['LM_Start_Wt_lf'] + 0.067 * row['1st_Probe_temp_lf'] + -2.6335 * row['Al_Bar_lf'] + -0.0373 * row['Lime_tap'] + -1883.7621 * row['AL_chem_first'] + 4583.8493 * row['S_chem_first']

            row['AL_mat'] = o

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] <= 0.024) and (row['AL_chem_first'] <= 0.035):

            row['Oxygen Node'] = 3

            o = 0.0394 * row['final_O2_ppm'] + 0.4446 * row['LM_Start_Wt_lf'] + 0.0591 * row['1st_Probe_temp_lf'] + -1.131 * row['Al_Bar_lf'] + -0.0342 * row['Lime_tap'] + -1805.4052 * row['AL_chem_first'] + 4872.0171 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        else:

            row['AL_mat'] = c

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] <= 0.01) and (row['S_chem_first'] <= 0.02):

        row['Carbon Node'] = 9

        c = 0.5809 * row['LM_Start_Wt_lf'] + 0.1717 * row['1st_Probe_temp_lf'] + 2.4264 * row['Al_Bar_lf'] + -0.0341 * row['Lime_tap'] + -3285.9989 * row['AL_chem_first'] + 3091.3112 * row['S_chem_first'] + -0.0017 * row['LTA_lf'] + -0.117 * row['SiMn_tap'] + -4937.5203 * row['SI_chem_first'] + -2871.3224 * row['C_chem_first']

        if (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] <= 0.01) and (row['S_chem_first'] <= 0.02) and (row['Lime_tap'] <= 1010.2):

            row['Oxygen Node'] = 12

            o = 0.0931 * row['final_O2_ppm'] + 0.5727 * row['LM_Start_Wt_lf'] + -0.0297 * row['1st_Probe_temp_lf'] + -1.7368 * row['Al_Bar_lf'] + 0.2081 * row['Lime_tap'] + 1476.5607 * row['AL_chem_first'] + -454.0645 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        else:

            row['AL_mat'] = c

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] <= 0.01) and (row['S_chem_first'] > 0.02):

        row['Carbon Node'] = 10

        c = 0.2572 * row['LM_Start_Wt_lf'] + 0.2498 * row['1st_Probe_temp_lf'] + -1.0351 * row['Al_Bar_lf'] + -0.0559 * row['Lime_tap'] + -3959.3532 * row['AL_chem_first'] + 4823.2363 * row['S_chem_first'] + 0.0095 * row['LTA_lf'] + -0.0664 * row['SiMn_tap'] + -8553.5977 * row['SI_chem_first'] + -2432.6962 * row['C_chem_first']

        if (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] <= 0.01) and (row['S_chem_first'] > 0.02) and (row['LM_Start_Wt_lf'] <= 203.743):

            row['Oxygen Node'] = 9

            o = -0.0114 * row['final_O2_ppm'] + -0.4109 * row['LM_Start_Wt_lf'] + 0.1836 * row['1st_Probe_temp_lf'] + 7.7806 * row['Al_Bar_lf'] + -0.086 * row['Lime_tap'] + 645.4803 * row['AL_chem_first'] + 3057.6657 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        else:

            row['AL_mat'] = c

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.021) and (row['S_chem_first'] > 0.029):

        row['Carbon Node'] = 11

        c = 0.7856 * row['LM_Start_Wt_lf'] + -0.0129 * row['1st_Probe_temp_lf'] + 4.362 * row['Al_Bar_lf'] + -0.0659 * row['Lime_tap'] + -1992.4572 * row['AL_chem_first'] + 7477.7345 * row['S_chem_first'] + -0.0007 * row['LTA_lf'] + 0.0208 * row['SiMn_tap'] + -2220.0319 * row['SI_chem_first'] + -1081.9221 * row['C_chem_first']

        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] <= 0.03):

            row['Oxygen Node'] = 5

            o = 0.0472 * row['final_O2_ppm'] + 0.4099 * row['LM_Start_Wt_lf'] + 0.0752 * row['1st_Probe_temp_lf'] + -1.4064 * row['Al_Bar_lf'] + -0.0045 * row['Lime_tap'] + -2538.0108 * row['AL_chem_first'] + 3874.9522 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] > 0.03):

            row['Oxygen Node'] = 13

            o = -0.0843 * row['final_O2_ppm'] + 1.9108 * row['LM_Start_Wt_lf'] + 0.0295 * row['1st_Probe_temp_lf'] + -5.8736 * row['Al_Bar_lf'] + -0.0418 * row['Lime_tap'] + -757.535 * row['AL_chem_first'] + 3494.3731 * row['S_chem_first']

            row['AL_mat'] = o

        else:

            row['AL_mat'] = c

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.016):

        row['Carbon Node'] = 12
        c = 0.1261 * row['LM_Start_Wt_lf'] + 0.1722 * row['1st_Probe_temp_lf'] + 1.0519 * row['Al_Bar_lf'] + -0.016 * row['Lime_tap'] + -1284.3628 * row['AL_chem_first'] + 4291.4388 * row['S_chem_first'] + -0.01 * row['LTA_lf'] + -0.0203 * row['SiMn_tap'] + -3953.4701 * row['SI_chem_first'] + -2227.1677 * row['C_chem_first']

        if (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.024) and (row['final_O2_ppm'] <= 1016.35):

            row['Oxygen Node'] = 14

            o = 0.0056 * row['final_O2_ppm'] + 0.8489 * row['LM_Start_Wt_lf'] + 0.1301 * row['1st_Probe_temp_lf'] + -5.7387 * row['Al_Bar_lf'] + -0.0643 * row['Lime_tap'] + -3757.1869 * row['AL_chem_first'] + 5313.2647 * row['S_chem_first']

            row['AL_mat'] = o

        else:

            row['AL_mat'] = c

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] > 0.024) and (row['AL_chem_first'] > 0.022):

        row['Carbon Node'] = 13
        c = 0.5743 * row['LM_Start_Wt_lf'] + 0.0822 * row['1st_Probe_temp_lf'] + -1.4686 * row['Al_Bar_lf'] + 0.0017 * row['Lime_tap'] + -1187.9995 * row['AL_chem_first'] + 4171.1597 * row['S_chem_first'] + 0.0015 * row['LTA_lf'] + -0.1217 * row['SiMn_tap'] + 1952.4868 * row['SI_chem_first'] + -1138.2887 * row['C_chem_first']

        if (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] <= 0.03):

            row['Oxygen Node'] = 5

            o = 0.0472 * row['final_O2_ppm'] + 0.4099 * row['LM_Start_Wt_lf'] + 0.0752 * row['1st_Probe_temp_lf'] + -1.4064 * row['Al_Bar_lf'] + -0.0045 * row['Lime_tap'] + -2538.0108 * row['AL_chem_first'] + 3874.9522 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] > 0.027) and (row['AL_chem_first'] <= 0.05) and (row['S_chem_first'] > 0.024) and (row['S_chem_first'] > 0.03):

            row['Oxygen Node'] = 13

            o = -0.0843 * row['final_O2_ppm'] + 1.9108 * row['LM_Start_Wt_lf'] + 0.0295 * row['1st_Probe_temp_lf'] + -5.8736 * row['Al_Bar_lf'] + -0.0418 * row['Lime_tap'] + -757.535 * row['AL_chem_first'] + 3494.3731 * row['S_chem_first']

            row['AL_mat'] = min(c,o)

        elif (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.024) and (row['final_O2_ppm'] > 1016.35):

            row['Oxygen Node'] = 10

            o = 0.0172 * row['final_O2_ppm'] + 0.7945 * row['LM_Start_Wt_lf'] + 0.0299 * row['1st_Probe_temp_lf'] + -0.0299 * row['Al_Bar_lf'] + -0.0134 * row['Lime_tap'] + -2280.7642 * row['AL_chem_first'] + 4619.1716 * row['S_chem_first']

            row['AL_mat'] = min(c,o) - 33

        elif (row['AL_chem_first'] <= 0.027) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] > 0.024) and (row['final_O2_ppm'] <= 1016.35):

            row['Oxygen Node'] = 14

            o = 0.0056 * row['final_O2_ppm'] + 0.8489 * row['LM_Start_Wt_lf'] + 0.1301 * row['1st_Probe_temp_lf'] + -5.7387 * row['Al_Bar_lf'] + -0.0643 * row['Lime_tap'] + -3757.1869 * row['AL_chem_first'] + 5313.2647 * row['S_chem_first']

            row['AL_mat'] = min(c,o) - 33

        else:

            row['AL_mat'] = c


    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] > 0.015) and (row['S_chem_first'] > 0.024) and (row['AL_chem_first'] <= 0.022):
        row['Carbon Node'] = 14
        c = -0.1188 * row['LM_Start_Wt_lf'] + 0.2324 * row['1st_Probe_temp_lf'] + 7.6551 * row['Al_Bar_lf'] + -0.1695 * row['Lime_tap'] + 5022.7992 * row['AL_chem_first'] + 666.7963 * row['S_chem_first'] + -0.0046 * row['LTA_lf'] + 0.1344 * row['SiMn_tap'] + -10060.0 * row['SI_chem_first'] + -2723.9295 * row['C_chem_first']

        o, node = cgcr_oxygen_tree(row)

        row['Oxygen Node'] = node
        row['AL_mat'] = min(c,o)

    elif (row['AL_chem_first'] > 0.03) and (row['AL_chem_first'] > 0.05) and (row['SI_chem_first'] <= 0.004) and (row['SiMn_tap'] > 170.119):

        row['Carbon Node'] = 15

        o, node = cgcr_oxygen_tree(row)

        row['Oxygen Node'] = node
        row['AL_mat'] = o

    elif (row['AL_chem_first'] <= 0.03) and (row['AL_chem_first'] <= 0.015) and (row['AL_chem_first'] > 0.01) and (row['S_chem_first'] <= 0.016):

        row['Carbon Node'] = 16

        c = -1.0917 * row['LM_Start_Wt_lf'] + 0.5701 * row['1st_Probe_temp_lf'] + -21.2452 * row['Al_Bar_lf'] + -0.0651 * row['Lime_tap'] + 476.5523 * row['AL_chem_first'] + 6669.1629 * row['S_chem_first'] + -0.0036 * row['LTA_lf'] + 0.1242 * row['SiMn_tap'] + -5196.6844 * row['SI_chem_first'] + -2299.668 * row['C_chem_first']

        o, node = cgcr_oxygen_tree(row)

        row['Oxygen Node'] = node
        row['AL_mat'] = min(c,o)

    else:
        print("weak node")
        row['Carbon Node'] = 0
        row['Oxygen Node'] = 0
        row['AL_mat'] = (10.0 * (row['LM_Start_Wt_lf'] + 1) * (al_lsl - row['AL_chem_first'] + 16 * fade_rate ))/0.6

    return [row['AL_mat'], row['Carbon Node'], row['Oxygen Node']]


In [253]:
sorted(ads_test['Carbon_Node_2'].unique())#.sort_values()


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [258]:
for node in sorted(ads_test['Carbon_Node_2'].unique()):
# for node in [10,11]:
    print('node',node)
    model_test_1=ads_test[(ads_test['Carbon_Node_2']==node) ]
#     model_test_1=ads_test[(ads_test['Carbon_Node_2']==3) | (ads_test['Carbon_Node_2']==5)]
    #model_test_1=ads_test.copy()
    print(len(model_test_1))

    model_test_1.fillna(0, inplace = True)

    # ads preprocessing

    model_test_1['Al_Bar_lf'] = model_test_1['Al_Bar_lf'].clip(15,25)
    model_test_1['LM_Start_Wt_lf'] = model_test_1['LM_Start_Wt_lf'].clip(170,205)
    model_test_1['LTA_lf'] = model_test_1['LTA_lf'].clip(60)
    model_test_1['1st_Probe_temp_lf'] = model_test_1['1st_Probe_temp_lf'].clip(1550)
    model_test_1['final_O2_ppm'] = model_test_1['final_O2_ppm'].clip(700,1200)
    #model_test_1['Lime_tap'] = model_test_1['Lime_tap'].clip(500)


    ################ AL Prediction #################################

    # predicting aluminium material
#     model_test_1['al_predictions'] =  model_test_1.apply(cgcr_aluminium_prediction_new, axis = 1)
    model_test_1['al_predictions'] =  model_test_1.apply(cgcr_aluminium_prediction_original, axis = 1)

    # splitting the list of outputs into separate columns
    model_test_1['AL_KGS'] = model_test_1['al_predictions'].apply(lambda x: x[0])
    model_test_1['Carbon Node'] = model_test_1['al_predictions'].apply(lambda x: x[1])
    model_test_1['Oxygen Node'] = model_test_1['al_predictions'].apply(lambda x:x[2])
    #ads_test['AL_KGS_2'] = ads_test['al_predictions'].apply(lambda x:x[3])
    del model_test_1['al_predictions']

    #par=[0.855,0.0838,-1.3642,-0.036,-1413.6468,4535.899,-0.0013,0.1752,-17950.0,-1058.9498]
    #ads_test['AL_KGS'] =  par[0]* ads_test['LM_Start_Wt_lf'] +  par[1]* ads_test['1st_Probe_temp_lf'] + par[2] * ads_test['Al_Bar_lf'] +  par[3]* ads_test['Lime_tap'] +  par[4]* ads_test['AL_chem_first'] +  par[5]* ads_test['S_chem_first'] +  par[6]* ads_test['LTA_lf'] + par[7] * ads_test['SiMn_tap'] + par[8] * ads_test['SI_chem_first'] +  par[9]* ads_test['C_chem_first']
    # replacing -ve predictions as 0
    model_test_1['AL_KGS'][model_test_1['AL_KGS']<0]=0

    # predicting aluminium material in meters
    model_test_1['AL_MTS'] = model_test_1['AL_KGS'] * (3)


    model_test_1=model_test_1.merge(actual_al[['HEAT_NUMBER','Actual Al']],on='HEAT_NUMBER', how='left')
    model_test_1['Exclude Flag']=0
    model_test_1['Exclude Flag'][model_test_1['Actual Al'].isna()]=1
    model_test_1['Exclude Flag'][model_test_1['Actual Al']==0]=1
    model_test_1['Actual Al'][model_test_1['Actual Al']==0]=1
    model_test_1['Actual Al']=model_test_1['Actual Al'].fillna(1)
    model_test_1['Delta']=abs(model_test_1['AL_KGS']-model_test_1['Actual Al'])
    model_test_1['MAPE']=model_test_1['Delta']/model_test_1['Actual Al']
    model_test_1['Delta_flag']=0
    model_test_1['Delta_flag'][model_test_1['Delta']<=33]=1
    model_test_1['Delta_flag_2']=0
    model_test_1['Delta_flag_2'][model_test_1['Delta']<=50]=1
    ads_mape=model_test_1[model_test_1['Exclude Flag']==0]
    mape=ads_mape['MAPE'].mean()
    delta=ads_mape['Delta'].mean()
    print(len(ads_mape),'heats @',delta)
    print('Mape',mape)
    print('tolerance=',ads_mape['Delta_flag'].sum()/len(ads_mape))
    print('tolerance @ 50=',ads_mape['Delta_flag_2'].sum()/len(ads_mape))

    model_test_1['al_addition_equation'] =  model_test_1.apply(al_addition_equation, axis = 1)
    model_test_1['al_addition_kg'] = model_test_1['al_addition_equation'].apply(lambda x: x[0])
    model_test_1['al_addition_s_kg'] = model_test_1['al_addition_equation'].apply(lambda x: x[1])
    del model_test_1['al_addition_equation']



node 0
1488
1392 heats @ 75.09645201644416
Mape 0.5243115970110946
tolerance= 0.3074712643678161
tolerance @ 50= 0.444683908045977
node 1
499
419 heats @ 119.62560439392033
Mape 0.6395072290259116
tolerance= 0.30787589498806683
tolerance @ 50= 0.4152744630071599
node 2
461
419 heats @ 137.0849364390294
Mape 0.6130665449779514
tolerance= 0.2171837708830549
tolerance @ 50= 0.28878281622911695
node 3
628
557 heats @ 59.62902676718322
Mape 0.3860530400375889
tolerance= 0.4272890484739677
tolerance @ 50= 0.63016157989228
node 4
48
46 heats @ 90.07054899565217
Mape 0.5645840003311143
tolerance= 0.32608695652173914
tolerance @ 50= 0.45652173913043476
node 5
559
522 heats @ 71.23421569537842
Mape 0.24639333759969323
tolerance= 0.40229885057471265
tolerance @ 50= 0.5613026819923371
node 6
185
173 heats @ 93.37339225071497
Mape 0.4536226755880281
tolerance= 0.2774566473988439
tolerance @ 50= 0.4046242774566474
node 7
164
150 heats @ 84.31803856655367
Mape 0.25172096401369554
tolerance= 0.3133333

In [262]:
model_test_1.to_excel(r'C:\Users\khera siddharth\Desktop\Desktop_Folder\Case Work\JSW_dolvi\DeS\Working files\Model Prediction\Feb 23\output\testing\iterations\all_heats_new eqn.xlsx')